In [9]:
import asyncio
import json
import os

import numpy as np
from dotenv import load_dotenv
from lightrag import LightRAG, QueryParam
from lightrag.kg.shared_storage import initialize_pipeline_status
from lightrag.llm.openai import openai_complete_if_cache, openai_embed
from lightrag.utils import EmbeddingFunc
from supabase import create_client

load_dotenv()


True

In [10]:
import nest_asyncio

nest_asyncio.apply()


In [20]:
import os

WORKING_DIR = "./singapore_acts"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)


async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], **kwargs
) -> str:
    return await openai_complete_if_cache(
        "mistral-small-latest",
        # "DeepSeek-V3",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("VOTUM_API_KEY"),
        base_url=os.getenv("MODEL_BASE_URL", "https://model.thevotum.com"),
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await openai_embed(
        texts,
        model="text-embedding-3-small",
        api_key=os.getenv("VOTUM_API_KEY"),
        base_url=os.getenv("MODEL_BASE_URL", "https://model.thevotum.com"),
    )


async def get_embedding_dim():
    test_text = ["This is a test sentence."]
    embedding = await embedding_func(test_text)
    print(embedding)
    embedding_dim = embedding.shape[1]
    return embedding_dim


# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)


async def initialize_rag():
    embedding_dimension = await get_embedding_dim()
    print(f"Detected embedding dimension: {embedding_dimension}")

    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=llm_model_func,
        embedding_func=EmbeddingFunc(
            embedding_dim=embedding_dimension,
            max_token_size=8192,
            func=embedding_func,
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()

    return rag

In [21]:
rag = await initialize_rag()

INFO: Process 2111274 Shared-Data already initialized (multiprocess=False)
INFO:nano-vectordb:Load (111, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './singapore_acts/vdb_entities.json'} 111 data
INFO:nano-vectordb:Load (82, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './singapore_acts/vdb_relationships.json'} 82 data
INFO:nano-vectordb:Load (7, 1536) data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': './singapore_acts/vdb_chunks.json'} 7 data
INFO: Process 2111274 storage namespace already initialized: [full_docs]
INFO: Process 2111274 storage namespace already initialized: [text_chunks]
INFO: Process 2111274 storage namespace already initialized: [llm_response_cache]
INFO: Process 2111274 storage namespace already initialized: [doc_status]
INFO: Process 2111274 storage namespace already initialized: [full_docs]
INFO: Process 2111274 storage names

[[ 2.4632184e-02 -1.6649945e-05 -1.3136659e-02 ...  1.3440572e-02
  -1.0804123e-02  6.8000634e-03]]
Detected embedding dimension: 1536


In [16]:
await rag.ainsert("""Infrastructure Protection Act 2017
Infrastructure Protection
(Protected Areas) (No. 6)
Order 2025
In exercise of the powers conferred by section 8(1) of the Infrastructure Protection Act 2017, the Minister for Home Affairs makes the following Order:
Citation and commencement
1.  This Order is the Infrastructure Protection (Protected Areas) (No. 6) Order 2025 and comes into operation on 9 April 2025.
Premises declared to be protected area
2.  The premises described in the second column of the Schedule are declared to be a protected area for the purposes of the Act, and the person specified in the first column of that Schedule is designated as the authority of the protected area.
Made on 27 March 2025.
PANG KIN KEONG
Permanent Secretary,
Ministry of Home Affairs,
Singapore.
[SPF/CPS/C/09/12/32; AG/LEGIS/SL/137B/2025/2]""")

In [17]:
await rag.ainsert_custom_chunks("""Infrastructure Protection Act 2017
Infrastructure Protection
(Protected Places) (No. 8)
Order 2025
In exercise of the powers conferred by section 14(1) of the Infrastructure Protection Act 2017, the Minister for Home Affairs makes the following Order:
Citation and commencement
1.  This Order is the Infrastructure Protection (Protected Places) (No. 8) Order 2025 and comes into operation on 9 April 2025.
Premises declared to be protected places
2.  The premises described in the second column of the Schedule are declared to be protected places for the purposes of the Act, and the person specified in the first column of that Schedule is designated as the authority of the corresponding protected place.
Made on 27 March 2025.
PANG KIN KEONG
Permanent Secretary,
Ministry of Home Affairs,
Singapore.
[SPF/CPS/C/09/12/33; AG/LEGIS/SL/137B/2025/1]""")

In [18]:
await rag.ainsert("""Infrastructure Protection
Act 2017
2020 REVISED EDITION
This revised edition incorporates all amendments up to and including 1 December 2021 and comes into operation on 31 December 2021
An Act to provide for the protection of certain areas, places and other premises in Singapore against security risks.
[18 December 2018]
PART 1
PRELIMINARY
Short title
1.  This Act is the Infrastructure Protection Act 2017.
Interpretation
2.—(1)  In this Act, unless the context otherwise requires —
“act of terrorism” means any of the following:
(a)	a terrorist bombing offence within the meaning of the Terrorism (Suppression of Bombings) Act 2007;
(b)	a terrorist act within the meaning of the Terrorism (Suppression of Financing) Act 2002;
“Appeal Advisory Board” means the Appeal Advisory Board established under section 63;
“approved security plan”, for a special development or a special infrastructure, means any security plan that is approved by the Commissioner and is in force for that special development or that special infrastructure under this Act;
“authorised officer”, for a protected area or a protected place, means —
(a)	a police officer;
(b)	an officer or a senior military expert of the Singapore Armed Forces;
(c)	a soldier of the Singapore Armed Forces who is deployed to guard that protected area or that protected place;
(d)	an officer or a member of a visiting force lawfully present in Singapore who is deployed to guard that protected area or that protected place under the terms of any treaty, agreement or arrangement to which the Government is a party;
(e)	a prison officer (as defined in section 2 of the Prisons Act 1933) who is deployed to guard that protected area or that protected place;
(f)	an immigration officer (as defined in section 2 of the Immigration Act 1959) who is deployed to guard that protected area or that protected place;
(g)	an auxiliary police officer who is deployed to guard that protected area or that protected place; or
(h)	a security officer who is authorised by the Commissioner of Police under section 6(1) and is deployed to guard that protected area or that protected place;
“authority” —
(a)	for a protected area, means the person who is designated as the authority of the protected area in the protected area order for the protected area; or
(b)	for a protected place, means the person who is designated as the authority of the protected place in the protected place order for the protected place;
“auxiliary police officer” has the meaning given by section 2(1) of the Police Force Act 2004;
“building” means any permanent or temporary building or structure;
“Commissioner” means the Commissioner of Infrastructure Protection appointed under section 5, and (subject to any condition or restriction imposed under that section) includes a Deputy Commissioner of Infrastructure Protection or an Assistant Commissioner of Infrastructure Protection appointed under that section;
“Commissioner of Police” means the Commissioner of Police appointed under section 6(1) of the Police Force Act 2004;
“critical asset”, for any premises, means any facility, system or equipment which, if damaged or destroyed, may have a debilitating impact on the functioning of the premises;
“document” has the meaning given by section 29 of the Penal Code 1871;
“major renovation”, for any premises, means —
(a)	in the case of any part of a building that contains a critical asset, or is a public place or is otherwise accessible to vehicles or vessels, any alteration, extension, repair, dismantling or demolition works carried out to the structure or glazing of that part;
(b)	any installation or relocation of a critical asset in the premises; or
(c)	any alteration, extension, dismantling or demolition works affecting the perimeter of the premises;
“member of a state force” means any of the following persons:
(a)	a police officer;
(b)	an auxiliary police officer;
(c)	an officer, a senior military expert or a soldier of the Singapore Armed Forces;
(d)	a member of the Singapore Civil Defence Force, as defined in section 2 of the Civil Defence Act 1986;
(e)	any public officer, or any officer in a class of public officers, authorised in writing by the Minister for the purposes of Part 6;
“new development” means any new development that is proposed to be or being constructed;
“occupier”, for any premises, means the person in occupation or having the charge, management or control of the premises;
“officer”, “senior military expert”, “serviceman” and “soldier”, in relation to the Singapore Armed Forces, have the meanings given by section 2(1) of the Singapore Armed Forces Act 1972;
“owner”  —
(a)	for the common property of a building erected on land comprised in a strata title plan, means the management corporation having control of the building;
(b)	for the limited common property of a building erected on land comprised in a strata title plan, means the subsidiary management corporation having control of that limited common property;
(c)	for the common property of a building comprising flats sold under Part 4 of the Housing and Development Act 1959, means —
(i)	in the case where a Town Council (established under the Town Councils Act 1988) is liable for the management of that common property, the Town Council; and
(ii)	in any other case, the Housing and Development Board (established under the Housing and Development Act 1959); and
(d)	for any other premises, includes any mortgagee in possession and any person for the time being receiving the rent for the premises whether on the person’s own account or as agent or trustee or as receiver, or who would receive the same if the premises were let to a tenant, and any person whose name is entered in the Valuation List prepared under section 10 of the Property Tax Act 1960 as the owner of the premises;
“personal property”, in relation to a person, means any thing worn or carried by the person or apparently in the immediate control of the person;
“police officer” has the meaning given by section 2(1) of the Police Force Act 2004;
“premises” includes —
(a)	any building;
(b)	any land, whether built on or not;
(c)	any body of water; and
(d)	any area or place, whether or not enclosed, and whether underground or underwater;
“protected area” means any premises in Singapore declared in a protected area order to be a protected area;
“protected area order” means an order made under section 8 declaring any premises to be a protected area, and includes the order as amended under section 11;
“protected infrastructure” means —
(a)	a protected area;
(b)	a protected place;
(c)	a special development;
(d)	a special infrastructure; or
(e)	any premises in respect of which a security directive under section 55 or a Minister’s order under section 58 is issued;
“protected place” means any premises in Singapore declared in a protected place order to be a protected place;
“protected place order” means an order made under section 14 declaring any premises to be a protected place, and includes the order as amended under section 17;
“public place” means any premises to which the public, or a section of the public, ordinarily has access, whether on payment, by invitation, as of right, or by express or implied permission;
“repealed Act” means the Protected Areas and Protected Places Act (Cap. 256, 2013 Revised Edition) repealed by this Act;
“responsible person”, for a special development or a special infrastructure, has the meaning given to that expression in section 32;
“security agency” means any person licensed as a security agency under the Private Security Industry Act 2007;
“security measure” includes any installation, equipment, works, action or procedure manufactured, used or designed to be used for the purposes of reducing a security risk;
“security officer” has the meaning given by section 13 of the Private Security Industry Act 2007;
“Singapore Armed Forces” has the meaning given by section 2(1) of the Singapore Armed Forces Act 1972;
“special development” means a new development designated as a special development under section 34;
“special infrastructure” means —
(a)	a special infrastructure described in section 40; or
(b)	any premises designated as a special infrastructure under section 41;
“specified surrounding area”, for a protected area or a protected place, means any surrounding area of that protected area or that protected place that is specified in the protected area order (for that protected area) or the protected place order (for that protected place) for the purposes of sections 27 and 28;
“specified works”  —
(a)	for a special development, means any works for or affecting the foundation, retaining structure, substructure or superstructure of any building to be or being constructed under the special development; or
(b)	for a special infrastructure, means —
(i)	any extension of the special infrastructure;
(ii)	any major renovation to the special infrastructure; or
(iii)	any other type of works prescribed by the Minister;
“temporary occupation permit”, for a special development or a special infrastructure, means a temporary occupation permit granted or a certificate of statutory completion issued under the Building Control Act 1989 for that special development or that special infrastructure;
“vehicle” means any vehicle whether mechanically propelled or otherwise;
“vessel” includes any ship, boat, craft (including submersible craft), floating platform or air‑cushioned vehicle, whether mechanically propelled or otherwise, that is used to navigate any body of water;
“visiting force” has the meaning given by section 2(1) of the Visiting Forces Act 1960.
(2)  In this Act, unless the context otherwise requires, a reference to any premises, building or protected infrastructure includes a reference to a part of the premises, building or protected infrastructure.
Application of this Act
3.—(1)  Except as provided in subsection (2), this Act binds the Government and applies to any premises owned or occupied by the Government.
(2)  Nothing in this Act renders the Government liable to prosecution for an offence.
(3)  To avoid doubt, no person is immune from prosecution for any offence under this Act by reason only that the person is an employee of or is engaged to provide services to the Government.
Operation of other written laws
4.—(1)  Nothing in this Act affects the operation of, or derogates from, the following written laws:
(a)	the Air Navigation Act 1966;
(b)	the Building Control Act 1989;
(c)	the Civil Defence Shelter Act 1997;
(d)	the Criminal Procedure Code 2010;
(e)	the Fire Safety Act 1993;
(f)	the Liquor Control (Supply and Consumption) Act 2015;
(g)	the Planning Act 1998;
(h)	the Police Force Act 2004;
(i)	the Private Security Industry Act 2007;
(j)	the Public Order Act 2009;
(k)	the Singapore Armed Forces Act 1972;
(l)	any other prescribed written law (or any part of that written law as prescribed).
(2)  To avoid doubt, nothing in this Act affects —
(a)	the exercise by a police officer or an auxiliary police officer of any power under the Criminal Procedure Code 2010, the Public Order Act 2009 or any other written law; or
(b)	the exercise by a serviceman of any power in accordance with section 201D of the Singapore Armed Forces Act 1972.
""")

In [23]:

query = """What is the difference between a protected area and a protected place?"""

response = await rag.aquery(query,QueryParam(mode='hybrid',))
print(response)


The key differences between **protected areas** and **protected places** under Singapore's **Infrastructure Protection Act 2017** are as follows:  

### **Legal Basis**  
- **Protected Areas**: Declared under **section 8(1)** of the Act via a **protected area order** (e.g., *Infrastructure Protection (Protected Areas) (No. 6) Order 2025*).  
- **Protected Places**: Declared under **section 14(1)** of the Act via a **protected place order** (e.g., *Infrastructure Protection (Protected Places) (No. 8) Order 2025*).  

### **Designation Authority**  
- Both are designated by the **Minister for Home Affairs**, but the legal provisions and orders differ.  

### **Security Measures**  
- **Protected Areas**: Typically involve higher-security premises (e.g., military installations, critical infrastructure).  
- **Protected Places**: May include other sensitive locations requiring protection but with potentially differing security protocols.  

### **Operational Details**  
- Both are listed i

In [7]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()   

import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("MODEL_BASE_URL", "https://model.thevotum.com"))

res = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

print(res)

response =  client.embeddings.create(
    input=["The food at the restaurant was delicious and the service was great!"],
    # model="cohere-embed"
    model="text-embedding-3-small"
)


print(response)






ChatCompletion(id='chatcmpl-BT1xKAKvdKxPiL6NjzuimhxS6VLBD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1746258730, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_7a53abb7a2', usage=CompletionUsage(completion_tokens=31, prompt_tokens=13, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, '